Loading the dataset csv's into data frames and printing the first 10 values in the data set for each data frame to ensure that the data is being read and stored correctly

In [7]:
import pandas as pd

dataset_1 = pd.read_csv("CO2 emission by countries.csv")
dataset_2 = pd.read_csv("Historical Tropical Storm.csv")
dataset_3 = pd.read_csv("global_temps.csv")

print("From Dataset 1: CO2 emission by countries")
print(dataset_1.head(10))
print("\n")

print("From Dataset 2: Historical Tropical Storm")
print(dataset_2.head(10))
print("\n")

print("From Dataset 3: global_temps")
print(dataset_3.head(10))
print("\n")

From Dataset 1: CO2 emission by countries
       Country Code Calling Code  Year  CO2 emission (Tons)  Population(2022)  \
0  Afghanistan   AF           93  1750                  0.0        41128771.0   
1  Afghanistan   AF           93  1751                  0.0        41128771.0   
2  Afghanistan   AF           93  1752                  0.0        41128771.0   
3  Afghanistan   AF           93  1753                  0.0        41128771.0   
4  Afghanistan   AF           93  1754                  0.0        41128771.0   
5  Afghanistan   AF           93  1755                  0.0        41128771.0   
6  Afghanistan   AF           93  1756                  0.0        41128771.0   
7  Afghanistan   AF           93  1757                  0.0        41128771.0   
8  Afghanistan   AF           93  1758                  0.0        41128771.0   
9  Afghanistan   AF           93  1759                  0.0        41128771.0   

       Area % of World Density(km2)  
0  652230.0      0.40%      

Cleaning the data by replacing nan values with approprate values for the specific column, by first identifying where the nan values are in the datasets

In [8]:
#Identify the number of missing values in the datasets
print("Missing values from Dataset 1: CO2 emission by countries")
print(dataset_1.isnull().sum())
print("\n")

print("Missing values from Dataset 2: Historical Tropical Storm")
print(dataset_2.isnull().sum())
print("\n")

print("Missing values from Dataset 3: global_temps")
print(dataset_3.isnull().sum())
print("\n")

Missing values from Dataset 1: CO2 emission by countries
Country                   0
Code                   2168
Calling Code           3523
Year                      0
CO2 emission (Tons)       0
Population(2022)       6504
Area                   4336
% of World             4336
Density(km2)           6504
dtype: int64


Missing values from Dataset 2: Historical Tropical Storm
FID           0
YEAR          0
MONTH         0
DAY           0
AD_TIME       0
BTID          0
NAME          0
LAT           0
LONG          0
WIND_KTS      0
PRESSURE      0
CAT           0
BASIN         0
Shape_Leng    0
dtype: int64


Missing values from Dataset 3: global_temps
Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     1
Jul     1
Aug     1
Sep     1
Oct     1
Nov     1
Dec     1
J-D     1
D-N     2
DJF     1
MAM     0
JJA     1
SON     1
dtype: int64




In [ ]:
#Replacing the nan values in the datasets for "CO2 emission by countries"

#Identifying the categories in the "Code" column so I can match the format when filling in the nan values
print("Unique Code Values")
print(dataset_1["Code"].unique())
print("n")

#Format 2 letters in all caps, if nan replace with "XX"
dataset_1["Code"] = dataset_1["Code"].fillna("XX") 


#Identifying the categories in the "Calling Code" column so I can match the format when filling in the nan values
print("Unique Calling Code Values")
print(dataset_1["Calling Code"].unique())
print("n")

#Format 2 - 4 characters, if nan replace with "000"
dataset_1["Calling Code"] = dataset_1["Calling Code"].fillna("000") 


#Following the fomat I saw by overview the column, the values are int, I'll replace the missing population values with the avarage value of the population columns
#Doing the same for the "Area" and "% of World"

find_avg= ["Population(2022)", "Area"]#, "% of World"]

#temp assigning the nan values to 0 for the summing
dataset_1[find_avg] = dataset_1[find_avg].fillna(0)

#casting to int
dataset_1[find_avg] = dataset_1[find_avg].astype(int)

agv = dataset_1[find_avg].mean()
dataset_1[find_avg] = dataset_1[find_avg].fillna(agv) 


# "% of World" and "Density(km2)"" seem to be of type string, replacing the nan values with "Unknown" <-- not feeling too sure about the method I used here
nan_to_unknown= ["% of World", "Density(km2)"]
dataset_1[nan_to_unknown] = dataset_1[nan_to_unknown].fillna("Unknown")


#Updating the dataset_1 csv
dataset_1.to_csv("CO2 emission by countries.csv", index= False)

#Testing if the change was made
print("Missing values from Dataset 1: CO2 emission by countries")
print(dataset_1.isnull().sum())
print("\n")


Unique Code Values
['AF' 'AL' 'DZ' 'AD' 'AO' 'AI' 'AQ' 'AG' 'AR' 'AM' 'AW' 'AU' 'AT' 'AZ'
 'BS' 'BH' 'BD' 'BB' 'BY' 'BE' 'BZ' 'BJ' 'BM' 'BT' 'BO' 'BA' 'BW' 'BR'
 'BN' 'BG' 'BF' 'BI' 'KH' 'CM' 'CA' 'CV' 'CF' 'TD' 'CL' 'CN' 'CX' 'CO'
 'KM' 'CG' 'CK' 'CR' 'CI' 'HR' 'CU' 'CW' 'CY' 'CZ' 'CD' 'DK' 'DJ' 'DM'
 'DO' 'EC' 'EG' 'SV' 'GQ' 'ER' 'EE' 'SZ' 'ET' 'FO' 'FJ' 'FI' 'FR' 'GF'
 'PF' 'GA' 'GM' 'GE' 'DE' 'GH' 'GR' 'GL' 'GD' 'GP' 'GT' 'GN' 'GW' 'GY'
 'HT' 'HN' 'HK' 'HU' 'IS' 'IN' 'ID' 'IR' 'IQ' 'IE' 'IL' 'IT' 'JM' 'JP'
 'JO' 'KZ' 'KE' 'KI' 'XK' 'KW' 'KG' 'LA' 'LV' 'LB' 'LS' 'LR' 'LY' 'LI'
 'LT' 'LU' 'MO' 'MG' 'MW' 'MY' 'MV' 'ML' 'MT' 'MH' 'MQ' 'MR' 'MU' 'YT'
 'MX' 'FM' 'MD' 'MN' 'ME' 'MS' 'MA' 'MZ' 'MM' 'XX' 'NR' 'NP' 'NL' 'NC'
 'NZ' 'NI' 'NE' 'NG' 'NU' 'KP' 'MK' 'NO' 'OM' 'PK' 'PW' 'PS' 'PA' 'PG'
 'PY' 'PE' 'PH' 'PL' 'PT' 'PR' 'QA' 'RO' 'RW' 'KN' 'LC' 'PM' 'VC' 'WS'
 'ST' 'SA' 'SN' 'RS' 'SC' 'SL' 'SG' 'SX' 'SK' 'SI' 'SB' 'SO' 'ZA' 'SS'
 'ES' 'LK' 'SD' 'SR' 'SE' 'CH' 'TJ' 'TH' 'TL' 'TG' 'TO' 'T

In [25]:
#Cleaning the global_temps dataset


#Filling in all nan values with the avarage value of their column
find_avg_value= ["Jun","Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "J-D", "D-N", "DJF", "JJA", "SON"]

#temp assigning the nan values to 0 for the summing
dataset_3[find_avg_value] = dataset_3[find_avg_value].fillna(0)

#casting to float
dataset_3[find_avg_value] = dataset_3[find_avg_value].astype(float)

agv_value = dataset_3[find_avg_value].mean()
dataset_3[find_avg_value] = dataset_3[find_avg_value].fillna(agv_value) 


#Updating the dataset_3 csv
dataset_3.to_csv("global_temps.csv", index= False)

#Testing if the change was made
print("Missing values from Dataset 3: global_temps")
print(dataset_3.isnull().sum())
print("\n")


Missing values from Dataset 3: global_temps
Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     0
Sep     0
Oct     0
Nov     0
Dec     0
J-D     0
D-N     0
DJF     0
MAM     0
JJA     0
SON     0
dtype: int64


